In [1]:
# new import
import datetime as dt
import numpy as np
import pandas as pd
import pandas_datareader as pdr
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10,6)
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import seaborn as sns

In [3]:
default_date = dt.date.isoformat(dt.date.today() - dt.timedelta(397))

In [4]:
# get data with default lookback of about a year (after trimming)
def get_data(symbol,date=default_date):
    data = pdr.get_data_yahoo(symbol, start=date)
    return data

def calc_vol(df, n=21):
    """
    calculate rolling window volatility
    (STD), and add related columns. Default is 21 day.
    """
    df['Return'] = np.log(df.Close).diff()
    df['Volatility'] = df['Return'].rolling(n).std()
    df['Change'] = df['Close'].diff()
    df['Exp_Change'] = (df['Volatility'] * df['Close']).shift(1)
    df['Magnitude'] = df['Change'] / df['Exp_Change']
    df['Abs_Magnitude'] = np.abs(df.Magnitude)

# calculate intraday volatility
def high_low(df):
    df['High_Low_Spread'] = (df['High'] - df['Low']) / df['Close']

# return dataframe containing data for expiration Fridays
def exp_friday(df):
    mask = np.where((df.index.day > 14) & 
                    (df.index.day < 22) & 
                    (df.index.dayofweek == 4), True, False)
    return df[mask]

# return a data frame containing data with large upward moves
def low_vol_duration(df):
    pd.set_option('mode.chained_assignment', None)
    df['Days<2sd'] = 0
    count = 0
    for row in range(len(df)):

        if df['Magnitude'].iloc[row] < 2:
            count += 1
            df['Days<2sd'].iloc[row] = count
        else:    
            df['Days<2sd'].iloc[row] = count
            count = 0
    return df[df.Magnitude >=2]

In [6]:
fb = round(get_data('FB'),2)
fb.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2018-09-14,162.84,160.34,161.72,162.32,21770400,162.32
2018-09-17,162.06,159.77,161.92,160.58,21005300,160.58
2018-09-18,161.76,158.87,159.39,160.30,22465200,160.30
2018-09-19,163.44,159.48,160.08,163.06,19629000,163.06
2018-09-20,166.45,164.47,164.50,166.02,18936000,166.02


In [7]:
calc_vol(fb)
high_low(fb)

In [10]:
fb.dropna(inplace=True)
len(fb)

252

In [15]:
exp_fri = exp_friday(fb)
exp_fri[['Return', 'Change', 'Magnitude', 'Close']]

,Return,Change,Magnitude,Close
Date,,,,
2018-10-19,-0.005632,-0.87,-0.296249,154.05
2018-11-16,-0.030491,-4.32,-1.286083,139.53
2018-12-21,-0.065438,-8.45,-2.581390,124.95
2019-01-18,0.011665,1.74,0.345045,150.04
2019-02-15,-0.008884,-1.45,-0.317567,162.50
2019-03-15,-0.024931,-4.19,-1.871505,165.98
2019-05-17,-0.009079,-1.69,-0.477066,185.30
2019-06-21,0.008459,1.61,0.338365,191.14
2019-07-19,-0.012126,-2.42,-1.114338,198.36


In [16]:
big_days = low_vol_duration(fb)

In [18]:
big_days[['Return', 'Volatility', 'Change', 'Days<2sd']]

,Return,Volatility,Change,Days<2sd
Date,,,,
2018-10-16,0.033689,0.018390,5.26,0
2018-12-26,0.078417,0.032876,10.12,47
2019-01-30,0.042300,0.021921,6.23,22
2019-01-31,0.102704,0.029882,16.27,0
2019-04-25,0.056848,0.015354,10.68,57
2019-10-15,0.030150,0.015316,5.61,119


In [19]:
fb[['Days<2sd']].tail()

,Days<2sd
Date,
2019-10-10,117
2019-10-11,118
2019-10-14,119
2019-10-15,119
2019-10-16,1
